In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from typing import Type
from simulator import CovertGenerator, DarkGenerator
import numpy as np

In [2]:
def ba_generator(min_n: int=20, max_n: int=30, m: int=2) -> Type[nx.classes.graph.Graph]:
    n = np.random.randint(max_n - min_n + 1) + min_n
    return nx.barabasi_albert_graph(n ,m)

def covert_generator(min_n: int=20, max_n: int=30, density: int=0.1):
    generator = CovertGenerator(min_n, max_n, density)
    generator.simulate()
    return generator.G

def dark_generator(min_n: int=20, max_n: int=30, density: int=0.1):
    generator = DarkGenerator(min_n, max_n, density)
    generator.simulate()
    return generator.G

def _911_generator(data_dir: str="./911/", file_name: str="g_0"):
    return nx.read_gml(data_dir + file_name)

def write_gml(G: Type[nx.classes.graph.Graph], output_dir: str, file_name: str) -> None:
    nx.write_gml(G, output_dir+file_name)
    
def read_gml(input_dir: str, file_name: str) -> Type[nx.classes.graph.Graph]:
    G = nx.read_gml(input_dir + file_name)
    return nx.relabel_nodes(G, lambda x: int(x))

In [50]:
output_dir = "./911/"
file_name = "g_0"
ba_graph = ba_generator()
write_gml(ba_graph, output_dir, file_name)
# G = read_gml(output_dir, file_name)

In [9]:
covert_graph = covert_generator()

In [7]:
dark_graph = dark_generator()

In [8]:
dark_graph

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25))

In [11]:
_911_generator()